In [0]:
#import findspark
#findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping


#spark = SparkSession.builder.appName("movielens").getOrCreate()

# Consultas sobre Movielens

## Schema de Movielens

![Schema](movielens_schema.png)

## ¿Cuántas películas calificó cada usuario?

Proceso:

1. Cargar la tabla ratings
1. Cargar la tabla movies
1. Unir ambas tablas
1. Agrupar por usuario
1. Contar las calificaciones


### Cargar las tablas

In [0]:
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("dbfs:/FileStore/tables/ratings.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

# Tabla Movies
movies_schema  = StructType(fields=[
    StructField("movieId",IntegerType(),True), 
    StructField("title",StringType(),True),
    StructField("genres",StringType(),True)
])

moviesDf = spark.read\
    .option("header", True)\
    .schema(movies_schema)\
    .csv("dbfs:/FileStore/tables/movies.csv")

moviesDf = moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|"))\
                        .drop('genres').withColumnRenamed("genresSplit","genres")\
                            .withColumn(\
                                "year",\
                                regexp_extract(\
                                           moviesDf["title"],\
                                           "^.+\(([0-9]+)\)$",\
                                           1)\
                                .cast(IntegerType()))\
                            .withColumn(\
                            "title_temp",\
                            regexp_extract(\
                                           moviesDf["title"],\
                                           "^(.+?) \([0-9]+\)$",\
                                           1))\
                            .drop('title')\
                        .withColumnRenamed("title_temp","title")

### Unir Ambas tablas

In [0]:
movie_ratingsDF = ratingsDf.join(moviesDf,on="movieId",how="inner")
movie_ratingsDF.show()

+-------+------+------+--------+--------------------+----+--------------------+
|movieId|userId|rating|    date|              genres|year|               title|
+-------+------+------+--------+--------------------+----+--------------------+
|      1|     1|   4.0|20000730|[Adventure, Anima...|1995|           Toy Story|
|      3|     1|   4.0|20000730|   [Comedy, Romance]|1995|    Grumpier Old Men|
|      6|     1|   4.0|20000730|[Action, Crime, T...|1995|                Heat|
|     47|     1|   5.0|20000730| [Mystery, Thriller]|1995|Seven (a.k.a. Se7en)|
|     50|     1|   5.0|20000730|[Crime, Mystery, ...|1995| Usual Suspects, The|
|     70|     1|   3.0|20000730|[Action, Comedy, ...|1996| From Dusk Till Dawn|
|    101|     1|   5.0|20000730|[Adventure, Comed...|1996|       Bottle Rocket|
|    110|     1|   4.0|20000730|[Action, Drama, War]|1995|          Braveheart|
|    151|     1|   5.0|20000730|[Action, Drama, R...|1995|             Rob Roy|
|    157|     1|   5.0|20000730|       [

### Agrupar por usuario

In [0]:
from pyspark.sql.functions import grouping

movie_ratingsDF_grouped_by_userid = movie_ratingsDF.groupBy("userId")

### Contar las calificaciones

In [0]:
movie_ratingsDF_grouped_by_userid.agg({"*": "count"}).collect()

Out[7]: [Row(userId=148, count(1)=48),
 Row(userId=463, count(1)=33),
 Row(userId=471, count(1)=28),
 Row(userId=496, count(1)=29),
 Row(userId=243, count(1)=36),
 Row(userId=392, count(1)=25),
 Row(userId=540, count(1)=42),
 Row(userId=31, count(1)=50),
 Row(userId=516, count(1)=26),
 Row(userId=85, count(1)=34),
 Row(userId=137, count(1)=141),
 Row(userId=251, count(1)=23),
 Row(userId=451, count(1)=34),
 Row(userId=580, count(1)=436),
 Row(userId=65, count(1)=34),
 Row(userId=458, count(1)=59),
 Row(userId=53, count(1)=20),
 Row(userId=255, count(1)=44),
 Row(userId=481, count(1)=31),
 Row(userId=588, count(1)=56),
 Row(userId=133, count(1)=35),
 Row(userId=296, count(1)=27),
 Row(userId=472, count(1)=29),
 Row(userId=78, count(1)=61),
 Row(userId=322, count(1)=107),
 Row(userId=513, count(1)=32),
 Row(userId=321, count(1)=56),
 Row(userId=362, count(1)=109),
 Row(userId=375, count(1)=33),
 Row(userId=593, count(1)=103),
 Row(userId=597, count(1)=443),
 Row(userId=108, count(1)=76),

In [0]:
# Alternativa: metodo count
movie_ratingsDF_grouped_by_userid.count().show(10)

+------+-----+
|userId|count|
+------+-----+
|   148|   48|
|   463|   33|
|   471|   28|
|   496|   29|
|   243|   36|
|   392|   25|
|   540|   42|
|    31|   50|
|   516|   26|
|    85|   34|
+------+-----+
only showing top 10 rows



Existen muchas funciones que se pueden utilizar en agregaciones, la lista está en [Grouping](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html)

## Filtrado

Podemos filtrar por cualquier columna.

Filtrado por usuario:

In [0]:
movie_ratingsDF.filter(movie_ratingsDF.userId == 148).show()

+-------+------+------+--------+--------------------+----+--------------------+
|movieId|userId|rating|    date|              genres|year|               title|
+-------+------+------+--------+--------------------+----+--------------------+
|    356|   148|   4.0|20161224|[Comedy, Drama, R...|1994|        Forrest Gump|
|   1197|   148|   3.0|20161224|[Action, Adventur...|1987| Princess Bride, The|
|   4308|   148|   4.0|20161224|[Drama, Musical, ...|2001|        Moulin Rouge|
|   4886|   148|   3.0|20161224|[Adventure, Anima...|2001|      Monsters, Inc.|
|   4896|   148|   4.0|20161224|[Adventure, Child...|2001|Harry Potter and ...|
|   4993|   148|   3.0|20161224|[Adventure, Fantasy]|2001|Lord of the Rings...|
|   5618|   148|   3.0|20161224|[Adventure, Anima...|2001|Spirited Away (Se...|
|   5816|   148|   4.0|20161224|[Adventure, Fantasy]|2002|Harry Potter and ...|
|   5952|   148|   3.0|20161224|[Adventure, Fantasy]|2002|Lord of the Rings...|
|   6377|   148|   3.0|20161224|[Adventu

In [0]:
movie_ratingsDF.filter("userId = 148").show()

+-------+------+------+--------+--------------------+----+--------------------+
|movieId|userId|rating|    date|              genres|year|               title|
+-------+------+------+--------+--------------------+----+--------------------+
|    356|   148|   4.0|20161224|[Comedy, Drama, R...|1994|        Forrest Gump|
|   1197|   148|   3.0|20161224|[Action, Adventur...|1987| Princess Bride, The|
|   4308|   148|   4.0|20161224|[Drama, Musical, ...|2001|        Moulin Rouge|
|   4886|   148|   3.0|20161224|[Adventure, Anima...|2001|      Monsters, Inc.|
|   4896|   148|   4.0|20161224|[Adventure, Child...|2001|Harry Potter and ...|
|   4993|   148|   3.0|20161224|[Adventure, Fantasy]|2001|Lord of the Rings...|
|   5618|   148|   3.0|20161224|[Adventure, Anima...|2001|Spirited Away (Se...|
|   5816|   148|   4.0|20161224|[Adventure, Fantasy]|2002|Harry Potter and ...|
|   5952|   148|   3.0|20161224|[Adventure, Fantasy]|2002|Lord of the Rings...|
|   6377|   148|   3.0|20161224|[Adventu

Mostrar solo ciertas columnas:

In [0]:
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .select("userId","title")\
    .show()

+------+--------------------+
|userId|               title|
+------+--------------------+
|   148|        Forrest Gump|
|   148| Princess Bride, The|
|   148|        Moulin Rouge|
|   148|      Monsters, Inc.|
|   148|Harry Potter and ...|
|   148|Lord of the Rings...|
|   148|Spirited Away (Se...|
|   148|Harry Potter and ...|
|   148|Lord of the Rings...|
|   148|        Finding Nemo|
|   148|Lord of the Rings...|
|   148|Harry Potter and ...|
|   148|Phantom of the Op...|
|   148|Howl's Moving Cas...|
|   148|   Pride & Prejudice|
|   148|Harry Potter and ...|
|   148|      V for Vendetta|
|   148|         Ratatouille|
|   148|Harry Potter and ...|
|   148|              WALL·E|
+------+--------------------+
only showing top 20 rows



Otras formas de agregar datos:

Calcular el promedio de datos

In [0]:
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .select(avg("rating"))\
    .show()


+-----------+
|avg(rating)|
+-----------+
|    3.73958|
+-----------+



Cual es el máximo rating que da el usuario 148?

In [0]:
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .select(max("rating"))\
    .show()

+-----------+
|max(rating)|
+-----------+
|        5.0|
+-----------+



En cuales peliculas?

In [0]:
# Encadenar Filtros
movie_ratingsDF\
    .filter(movie_ratingsDF.userId == 148)\
    .filter(movie_ratingsDF.rating == 5.0)\
    .select(movie_ratingsDF.title)\
    .show()

+--------------------+
|               title|
+--------------------+
|Phantom of the Op...|
|   Pride & Prejudice|
|            Paperman|
+--------------------+



In [0]:
#Sintaxis SQL

movie_ratingsDF\
    .filter("userId = 148 AND rating = 5.0")\
    .select(movie_ratingsDF.title)\
    .show()

+--------------------+
|               title|
+--------------------+
|Phantom of the Op...|
|   Pride & Prejudice|
|            Paperman|
+--------------------+



### Pivot

Algo muy poderoso son las *pivot tables* que permiten analizar datos facilmente.

In [0]:
movie_ratingsDF.filter("movieId = 1197 AND date <= '19970930'").groupBy("userId").pivot("date").agg(avg("rating")).show()

+------+--------+--------+--------+--------+--------+--------+--------+--------+
|userId|19961213|19961226|19970223|19970319|19970330|19970530|19970621|19970916|
+------+--------+--------+--------+--------+--------+--------+--------+--------+
|    31| 5.00000|    null|    null|    null|    null|    null|    null|    null|
|   385|    null|    null|    null|    null|    null| 4.00000|    null|    null|
|   372|    null|    null|    null|    null|    null|    null|    null| 4.00000|
|   437|    null|    null|    null|    null| 4.00000|    null|    null|    null|
|    84|    null|    null|    null| 4.00000|    null|    null|    null|    null|
|   171|    null|    null|    null|    null|    null|    null| 4.00000|    null|
|   524|    null| 5.00000|    null|    null|    null|    null|    null|    null|
|    32|    null|    null| 5.00000|    null|    null|    null|    null|    null|
+------+--------+--------+--------+--------+--------+--------+--------+--------+

